In [1]:
# load the functionality into Julia
using StatsFuns, CSV, Cubature, ProgressMeter, DataFrames, Optim, SpecialFunctions, Roots
using Distributions, DelimitedFiles, Random

using Plots, StatsPlots, ColorBrewer
pyplot()

Plots.PyPlotBackend()

In [2]:
Plots.scalefontsizes(1.2)

In [3]:
clrs = [ColorBrewer.palette("Blues", 5)[2], ColorBrewer.palette("Blues", 5)[4]];
clrs

In [4]:
# output file
results_dir = "../../results/Andrei"
results_file = "$(results_dir)/output.csv"
run(`mkdir -p $(results_dir)`)

Process(`mkdir -p ../../results/Andrei`, ProcessExited(0))

In [5]:
## The function is adopted from:
## from https://github.com/robertfeldt/BlackBoxOptim.jl/blob/master/src/utilities/latin_hypercube_sampling.jl
"""
    latin_hypercube_sampling(mins, maxs, numSamples)
    Randomly sample `numSamples` values from the parallelogram defined
    by `mins` and `maxs` using the Latin hypercube algorithm.
"""
function latin_hypercube_sampling(mins::AbstractVector{T},
                                  maxs::AbstractVector{T},
                                  n::Integer) where T<:Number
    length(mins) == length(maxs) ||
        throw(DimensionMismatch("mins and maxs should have the same length"))
    all(xy -> xy[1] <= xy[2], zip(mins, maxs)) ||
        throw(ArgumentError("mins[i] should not exceed maxs[i]"))
    dims = length(mins)
    result = zeros(T, dims, n)
    cubedim = Vector{T}(undef, n)
    @inbounds for i in 1:dims
        imin = mins[i]
        dimstep = (maxs[i] - imin) / n
        for j in 1:n
            cubedim[j] = imin + dimstep * (j - 1 + rand(T))
        end
        result[i, :] .= shuffle!(cubedim)
    end
    return result
end

nsamples = 10000
# for reproducibility we fix the seed of the random number generator
smpls = latin_hypercube_sampling([0.0],[1.0],nsamples);

# Distribution

In [6]:
weibullpdf(α, θ, x) = begin z = x / θ; (α / θ) * z^(α - 1) * exp(-z^α) end
weibullcdf(α, θ, x) = x > 0 ? -expm1(- ((x / θ) ^ α)) : 0.0

weibullcdf (generic function with 1 method)

# Without truncation

In [1]:
filenames = ["data_incper", "data_incper_inclwuhan", "data_ons_hosp", "dthdata_hosp_dth", "dthdata_ons_dth", "dthdata_ons_hosp"]
tmax = [16, 16, 16, 35, 35, 16]
tmstps = [4, 4, 4, 7, 7, 7]

logmeans_inits = log.([4.2, 3, 3.7, 5., 6.2, 7.2])
logα_inits = log.([1.05, 1.2, 1.2, 1.4, 1.4, 2.4])

idx = 1
for flname in filenames[idx:end]
    println(flname)
    df = DataFrame(CSV.read("../../data/$(flname).csv", missingstring="NA"))
    select!(df, [:EL, :ER, :SL, :SR, :tstar])
    rename!(df, Dict(:EL=>:E_L, :ER=>:E_R, :SL=>:S_L, :SR=>:S_R))
    
    if (idx==2)
        df[df[!,:E_L].==0,:E_L] .= df[df[!,:E_L].==0,:S_L] .- 28.
    end
    
    prog = ProgressThresh(-Inf, "Without truncation:") 
    function getNegativeLoglk(params)
        mean_ = exp(params[1]) # make the parameters positive
        α = exp(params[2]) # make the parameters positive
        θ = mean_ / gamma(1 + 1/α)
        loglk = 0
        @inline function fs(t) 
            weibullpdf(α, θ, t)
        end
        @inline function Fs(t) 
            weibullcdf(α, θ, t)
        end
        # this for-loop is based on coarseData R package consolidated to one piece of code
        @simd for k in 1:nrow(df)
            E_R = df[k,:E_R]; E_L = df[k,:E_L]; S_R = df[k,:S_R]; S_L = df[k,:S_L]
            if (S_L!=S_R)
                if (S_L > E_R) # if the intervals do not overlap
                    loglk += log(hquadrature(e -> (Fs(S_R-e) - Fs(S_L-e)), E_L, E_R)[1])
                elseif (S_L > E_L)
                    loglk += log(hquadrature(e -> (Fs(S_R-e) - Fs(S_L-e)), E_L, S_L)[1] + 
                                 hquadrature(e -> Fs(S_R-e), S_L, E_R)[1])
                else
                    loglk += log(hquadrature(e -> Fs(S_R-e), E_L, E_R)[1])
                end
            else
                if (S_R > E_R) # if the intervals do not overlap
                    loglk += log(hquadrature(e -> fs(S_R-e), E_L, E_R)[1])
                elseif (S_L > E_L)
                    loglk += log(hquadrature(e -> fs(S_R-e), E_L, S_R)[1] + hquadrature(e -> fs(S_R-e), S_R, E_R)[1])
                end
            end
        end
        ProgressMeter.update!(prog, -loglk)
        return(-loglk)
    end
    
    println("* MLE")
    res0 = optimize(x -> getNegativeLoglk(x), [logmeans_inits[idx], logα_inits[idx]], LBFGS())
    
    println("* Generating the results")
    logprms = Optim.minimizer(res0)
    prms = exp.(logprms)
    α = prms[2] # make the parameters positive
    θ = prms[1] / gamma(1 + 1.0/prms[2])
    println("* for note: $(α), $(θ)")
    logmeans_inits[idx] = logprms[1]
    logα_inits[idx] = logprms[2]
    mean_ = prms[1]
    α_ = prms[2]
    sd_ = mean_ * sqrt(gamma(1.0 + 2.0/α_)/gamma(1.0 + 1.0/α_)^2 - 1.0)

    negloglk = Optim.minimum(res0)
    AIC = -2*2 + 2negloglk

    res = [mean_, sd_, negloglk, AIC]
    
    println("* profile-based for the mean")
    step = .1
    y = x -> getNegativeLoglk([x, logprms[2]]) - negloglk - 1.92
    x0 = logprms[1]
    while y(x0)<0
        x0 -= step
    end
    logprms1_lower = fzero(y, (x0,logprms[1]))[1]

    x0 = logprms[1]
    while y(x0)<0
        x0 += step
    end
    logprms1_upper = fzero(y, (logprms[1], x0))[1]

    res_ = exp.([logprms1_lower, logprms1_upper])
    res = vcat(res, res_)
    means_samples = latin_hypercube_sampling([res_[1]],[res_[2]],nsamples);
    
    println("* profile-based for the SD")
    step = .1
    y = x -> getNegativeLoglk([logprms[1], x]) - negloglk - 1.92
    x0 = logprms[2]
    while y(x0)<0
        x0 -= step
    end
    logprms2_lower = fzero(y, (x0,logprms[2]))[1]

    x0 = logprms[2]
    while y(x0)<0
        x0 += step
    end
    logprms2_upper = fzero(y, (logprms[2], x0))[1]

    res_ = exp.([logprms2_lower, logprms2_upper])
    α_samples = latin_hypercube_sampling([res_[1]],[res_[2]],nsamples);
    
    SD_samples = []
    for idx in 1:nsamples
        append!(SD_samples,[means_samples[idx]*sqrt(gamma(1.0+2.0/α_samples[idx])/(gamma(1.0+1.0/α_samples[idx]))^2-1.0)])
    end
    res_ = [minimum(SD_samples), maximum(SD_samples)]
    res = vcat(res, res_)
    
    println("* Creating a plot")
    t = [0:0.05:tmax[idx];]
    for idx in 1:nsamples
        α_ = α_samples[idx]
        θ_ = means_samples[idx] / gamma(1 + 1/α_)
        if idx>1
            append!(y,[cdf.(Weibull(α_, θ_),t)])
        else 
            y = [cdf.(Weibull(α_, θ_),t)]
        end
    end
    plot(t, y[1:100], legend=false, color=clrs[1], alpha=.3, 
        grid=false, yticks = 0:0.2:1, xticks = 0:tmstps[idx]:tmax[idx],
        xlabel="Exposure to onset (days)", ylabel="Proportion with symptoms",
        size = (300,400), dpi = 300)
    plot!(t, median(hcat(y...);dims=2), color=clrs[2], lw=1.4)

    quantiles = [0.05,.5,.95,.99]
    for idx in 1:nsamples
        α_ = α_samples[idx]
        θ_ = means_samples[idx] / gamma(1 + 1/α_)
        if idx>1
            append!(y,[quantile.(Weibull(α_, θ_),quantiles)])
        else 
            y = [quantile.(Weibull(α_, θ_),quantiles)]
        end
    end
    y = hcat(y...)
    for i in 1:length(quantiles)
        if i<=3
            plot!(quantile(y[i,:],[0.025,.975]),[quantiles[i],quantiles[i]], color=clrs[2])
        end
    end
    scatter!(median(y;dims=2)[1:(end-1)], quantiles[1:(end-1)], 
        color=clrs[2], markercolor=clrs[2], markersize=5) 

    Plots.pdf("$(results_dir)/weibull-$(flname)-no_truncation.pdf")
    
    for i in 1:length(quantiles)
        res_ = quantile(y[i,:],[0.025, .5, .975])
        res = vcat(res, res_)
    end
    
    println(res)
    
    res=reshape(vcat(["$(flname)","Weibull", "no"],round.(res,digits=4)),1,length(res)+3)
    open(results_file, "a") do io
        writedlm(io, res, ',')
    end
    
    idx += 1
end

data_incper


UndefVarError: UndefVarError: CSV not defined

# With truncation

In [18]:
filenames = ["data_incper", "data_incper_inclwuhan", "data_ons_hosp", "dthdata_hosp_dth", "dthdata_ons_dth", "dthdata_ons_hosp"]
tmax = [16, 16, 16, 35, 35, 16]
tmstps = [4, 4, 4, 7, 7, 7]

logmeans_inits = log.([4.2, 9.2, 3.7, 4., 4., 5.])
logα_inits = log.([1.05, 2.12, 1.2, 1., 1., 1.1])
#[9.222521342198092, 2.123142000905211]

idx = 2
for flname in filenames[idx:end]
    println(flname)
    df = DataFrame(CSV.read("../../data/$(flname).csv", missingstring="NA"))
    select!(df, [:EL, :ER, :SL, :SR, :tstar])
    rename!(df, Dict(:EL=>:E_L, :ER=>:E_R, :SL=>:S_L, :SR=>:S_R))
    
    prog = ProgressThresh(-Inf, "With truncation:") 
    function getNegativeLoglk(params)
        if (idx == 2)
            df[df[!,:E_L].==0,:E_L] .= df[df[!,:E_L].==0,:E_R] .- 14
        else
            df[df[!,:E_L].==0,:E_L] .= df[df[!,:E_L].==0,:E_R] .- 28
        end
        r = 0.14
        mean_ = exp(params[1]) # make the parameters positive
        α = exp(params[2]) # make the parameters positive
        θ = mean_ / gamma(1 + 1/α)
        loglk = 0
        @inline function fs(t) 
            weibullpdf(α, θ, t)
        end
        @inline function Fs(t) 
            weibullcdf(α, θ, t)
        end
        # this for-loop is based on coarseData R package consolidated to one piece of code
        @simd for k in 1:nrow(df)
            E_R = df[k,:E_R]; E_L = df[k,:E_L]; S_R = df[k,:S_R]; S_L = df[k,:S_L]; tstar = df[k,:tstar]
            @inline function g(e)
                hquadrature(x -> r*exp(-r*x)/(1-exp(-r*x))*Fs(tstar-e-x), 0.001, tstar-e)[1]
            end
            if (S_L!=S_R)
                if (S_L > E_R) # if the intervals do not overlap
                    loglk += log(hquadrature(e -> (Fs(S_R-e) - Fs(S_L-e))/g(e), E_L, E_R)[1])
                elseif (S_L > E_L)
                    loglk += log(hquadrature(e -> (Fs(S_R-e) - Fs(S_L-e))/g(e), E_L, S_L)[1] + 
                                 hquadrature(e -> Fs(S_R-e)/g(e), S_L, E_R)[1])
                else
                    loglk += log(hquadrature(e -> Fs(S_R-e)/g(e), E_L, E_R)[1])
                end
            else
                if (S_R > E_R) # if the intervals do not overlap
                    loglk += log(hquadrature(e -> fs(S_R-e)/g(e), E_L, E_R)[1])
                elseif (S_L > E_L)
                    loglk += log(hquadrature(e -> fs(S_R-e)/g(e), E_L, S_R)[1] + hquadrature(e -> fs(S_R-e)/g(e), S_R, E_R)[1])
                end
            end
        end
        ProgressMeter.update!(prog, -loglk)
        return(-loglk)
    end
    
    println("* MLE")
    res0 = optimize(x -> getNegativeLoglk(x), [logmeans_inits[idx], logα_inits[idx]])
    
    println("* Generating the results")
    logprms = Optim.minimizer(res0)
    prms = exp.(logprms)
    println("* for note: $(prms)")
    logmeans_inits[idx] = logprms[1]
    logα_inits[idx] = logprms[2]
    mean_ = prms[1]
    α_ = prms[2]
    sd_ = mean_ * sqrt(gamma(1.0 + 2.0/α_)/gamma(1.0 + 1.0/α_)^2 - 1.0)

    negloglk = Optim.minimum(res0)
    AIC = -2*2 + 2negloglk

    res = [mean_, sd_, negloglk, AIC]
    
    println("* profile-based for the mean")
    step = .1
    y = x -> getNegativeLoglk([x, logprms[2]]) - negloglk - 1.92
    x0 = logprms[1]
    while y(x0)<0
        x0 -= step
    end
    logprms1_lower = fzero(y, (x0,logprms[1]))[1]

    x0 = logprms[1]
    while y(x0)<0
        x0 += step
    end
    logprms1_upper = fzero(y, (logprms[1], x0))[1]

    res_ = exp.([logprms1_lower, logprms1_upper])
    res = vcat(res, res_)
    means_samples = latin_hypercube_sampling([res_[1]],[res_[2]],nsamples);
    
    println("* profile-based for the SD")
    step = .1
    y = x -> getNegativeLoglk([logprms[1], x]) - negloglk - 1.92
    x0 = logprms[2]
    while y(x0)<0
        x0 -= step
    end
    logprms2_lower = fzero(y, (x0,logprms[2]))[1]

    x0 = logprms[2]
    while y(x0)<0
        x0 += step
    end
    logprms2_upper = fzero(y, (logprms[2], x0))[1]

    res_ = exp.([logprms2_lower, logprms2_upper])
    α_samples = latin_hypercube_sampling([res_[1]],[res_[2]],nsamples);
    
    SD_samples = []
    for idx in 1:nsamples
        append!(SD_samples,[means_samples[idx]*sqrt(gamma(1.0+2.0/α_samples[idx])/(gamma(1.0+1.0/α_samples[idx]))^2-1.0)])
    end
    res_ = [minimum(SD_samples), maximum(SD_samples)]
    res = vcat(res, res_)
    
    println("* Creating a plot")
    t = [0:0.05:tmax[idx];]
    for idx in 1:nsamples
        α_ = α_samples[idx]
        θ_ = means_samples[idx] / gamma(1 + 1/α_)
        if idx>1
            append!(y,[cdf.(Weibull(α_, θ_),t)])
        else 
            y = [cdf.(Weibull(α_, θ_),t)]
        end
    end
    plot(t, y[1:100], legend=false, color=clrs[1], alpha=.3, 
        grid=false, yticks = 0:0.2:1, xticks = 0:tmstps[idx]:tmax[idx],
        xlabel="Exposure to onset (days)", ylabel="Proportion with symptoms",
        size = (300,400), dpi = 300)
    plot!(t, median(hcat(y...);dims=2), color=clrs[2], lw=1.4)

    quantiles = [0.05,.5,.95,.99]
    for idx in 1:nsamples
        α_ = α_samples[idx]
        θ_ = means_samples[idx] / gamma(1 + 1/α_)
        if idx>1
            append!(y,[quantile.(Weibull(α_, θ_),quantiles)])
        else 
            y = [quantile.(Weibull(α_, θ_),quantiles)]
        end
    end
    y = hcat(y...)
    for i in 1:length(quantiles)
        if i<=3
            plot!(quantile(y[i,:],[0.025,.975]),[quantiles[i],quantiles[i]], color=clrs[2])
        end
    end
    scatter!(median(y;dims=2)[1:(end-1)], quantiles[1:(end-1)], 
        color=clrs[2], markercolor=clrs[2], markersize=5) 

    Plots.pdf("$(results_dir)/weibull-$(flname)-with_truncation.pdf")
    
    for i in 1:length(quantiles)
        res_ = quantile(y[i,:],[0.025, .5, .975])
        res = vcat(res, res_)
    end
    
    println(res)
    
    res=reshape(vcat(["$(flname)","Weibull", "yes"],round.(res,digits=4)),1,length(res)+3)
    open(results_file, "a") do io
        writedlm(io, res, ',')
    end
    
    idx += 1
end

data_incper_inclwuhan
* MLE


With truncation: (thresh = -Inf, value = 398.408)

* Generating the results
* for note: [7.233543859502653, 1.8368297650563943]
* profile-based for the mean


With truncation: (thresh = -Inf, value = 400.328)

* profile-based for the SD


...

* Creating a plot
[13.04004471932511, 8.374235517845158, 314.1620745481801, 624.3241490963602, 9.102561252868593, 26.903835278408067, 4.718411762727034, 22.09618542703941, 1.1878844764556282, 3.0334771081365, 6.254644129553458, 8.388437500294026, 15.929916342352168, 24.017534164943946, 20.40102311434441, 39.34555449124248, 63.24124780867276, 25.666987376560538, 51.249218667091256, 87.68887449219204]
dthdata_ons_dth
* MLE


With truncation: (thresh = -Inf, value = 282.733)

* Generating the results
* for note: [1.3808687941204297e22, 1.8645931585033746]
* profile-based for the mean


With truncation: (thresh = -Inf, value = 282.733)

InterruptException: InterruptException: